In [22]:
import dotenv
from openai import AzureOpenAI
from swarm import Swarm, Agent
import json
import re

In [2]:
dotenv.load_dotenv()
Azure_client = AzureOpenAI()
client = Swarm(client=Azure_client)

In [40]:
def query_project_data(project_id:str):
    """
    query project information by project_id
    """
    print(f"project id is {project_id}")
    # query
    return {'project_name':'匿名项目','status':'running'}

def format_msg(msg):
    """
    format project info by fields in info_dict object
    """
    msg_re = re.search(r"```json([\s\S]+?)```", msg,re.S)
    print(msg_re)
    if msg_re:
        info_dict = json.loads(msg_re.group(1))
        return f"""
        Hi,xxx:
        项目信息如下：

        项目名称：{info_dict.get('project_name')}
        项目状态：{info_dict.get('status')}
        
        祝好！
        """
    else:
        return """未发现项目信息"""

def send_email(msg):
    """
    send email to notify
    """
    print(msg)
    return "已发送"

query_agent = Agent(
    name="项目信息查询",
    instructions="""
    你是一个项目信息查询助手，当被问到项目信息查询时：
    使用query_project_data函数查询信息
    直接返回结果的json格式，不要添加其他内容
    """,
    functions=[query_project_data],
)

def run_query_func(msg: str) -> str:
    """
    查询项目信息并发送邮件
    """
    messages = [{"role":'user', "content": msg}]
    response = client.run(agent=query_agent, messages=messages)
    msg = response.messages[-1]["content"]
    print(msg)
    formated_msg = format_msg(msg)
    if formated_msg == "未发现项目信息":
        return formated_msg
    else:
        resp = send_email(formated_msg)
        print(resp)
        return resp

In [41]:
run_query_func("查询一下项目X101SCXXX-J001的信息")

project id is X101SCXXX-J001
```json
{
    "project_name": "匿名项目",
    "status": "running"
}
```
<re.Match object; span=(0, 67), match='```json\n{\n    "project_name": "匿名项目",\n    "sta>

        Hi,xxx:
        项目信息如下：

        项目名称：匿名项目
        项目状态：running
        
        祝好！
        
已发送


'已发送'